In [1]:
import os, json
import sys
import pandas as pd
import requests
from io import StringIO
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pd.options.plotting.backend = 'plotly'


sys.path.append('..')

import src

MainProcess


In [ ]:
import sqlite3
conn = sqlite3.connect(src.constants.SOURCE.DATA.DB.str)
Employment = pd.read_sql_query('SELECT * FROM Employment', conn)
GDP = pd.read_sql_query('SELECT * FROM GDP', conn)

In [11]:
def charts_Interest(variation:str)->go.Figure:
    Interest = pd.read_sql_query('SELECT * FROM Interest', conn)
    return (
        Interest
        .replace(r'^\.+$','0.0', regex=True)
        .astype(float)
        .plot(x = 'Year', y = variation, title = variation, kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )


charts_Interest('General savings deposits Nominal interest, % per year')

In [14]:

def charts_grossDomesticProduct(value_unit:str,Category)->go.Figure:
    return (
        pd.read_sql_query('SELECT * FROM GDP', conn)
        .set_index(['Value unit','Category'])
        .loc[pd.IndexSlice[value_unit, Category,:], :]#.T
        .reset_index().T
        .iloc[2:].reset_index(names = 'Quarter')
        .rename(columns = {0:Category})
        .assign(**{Category: lambda x: x[Category].astype(float)})
        .plot(x='Quarter', y=Category, title = f"{Category} in {value_unit}", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)   
        .update_yaxes(title_text = '')
    )

charts_grossDomesticProduct(value_unit= 'Current prices', Category = '1. Private final consumption')

In [16]:
def charts_consumerPriceIndex(variation:str)->go.Figure:
    return (
        pd.read_sql_query('SELECT * FROM CPI', conn)
        .plot(x = 'Month', y = variation, title = f"{variation} in Iceland", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )
charts_consumerPriceIndex(variation = 'Consumer price index Index')

In [18]:

def charts_EmploymentBySector()->go.Figure:
    return (
        pd.read_sql_query('SELECT * FROM Employment', conn)
        .pipe(lambda s: s.set_index(s.columns[:3].tolist()))
        .loc[pd.IndexSlice['Jobs','Total employment'],]
        .drop('Total - All activities')
        .T
        .reset_index(names = 'time')
        .melt(id_vars=['time'], var_name ='sector', value_name='Number of Jobs')
        .plot(x = 'time', y = 'Number of Jobs', color = 'sector',kind = 'area')
        .update_layout(showlegend = False)
    )   

charts_EmploymentBySector()


C:\Users\andnolan\AppData\Local\Temp\3\ipykernel_3312\891832913.py:3: PerformanceWarning:

indexing past lexsort depth may impact performance.

